In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_bom_winare
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH OLALA AS (
    SELECT EXPLODE(ARRAY('10184193', '10184194', '10243668', '10243669', '10243670', '10243671')) AS material_no
),
BOM AS (
    SELECT 
        CAST(material_no AS STRING) AS material_no, 
        base_uom, 
        base_quantity, 
        IFNULL(CASE WHEN CAST(bom_component AS DOUBLE) = CAST(CAST(bom_component AS DOUBLE) AS BIGINT) THEN CAST(CAST(bom_component AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(bom_component AS DECIMAL(38,10)))) END, "") AS bom_component,
        component_quantity, 
        component_uom
    FROM {catalog_name}.udp_wcm_silver_sap_car.master_bom
    WHERE 
        proc_date IS NOT NULL
        AND sapi_doc_no IN
            (
            SELECT DISTINCT sapi_doc_no 
            FROM {catalog_name}.udp_wcm_silver_sap_car.master_bom
            WHERE proc_date IS NOT NULL AND bom_component IS NOT NULL
            QUALIFY ROW_NUMBER() OVER (PARTITION BY material_no ORDER BY proc_date DESC) = 1
            )
),
BOM_OLALA AS (
    SELECT
        a.material_no AS product_id, a.base_uom, a.base_quantity,
        CASE WHEN UPPER(c.product_name) LIKE 'DUMMY%' OR UPPER(c.product_name) LIKE 'DUMY%' THEN b.material_no ELSE a.bom_component END AS component_id,
        CASE WHEN UPPER(c.product_name) LIKE 'DUMMY%' OR UPPER(c.product_name) LIKE 'DUMY%' THEN c1.product_name ELSE c.product_name END AS component_name,
        CASE WHEN UPPER(c.product_name) LIKE 'DUMMY%' OR UPPER(c.product_name) LIKE 'DUMY%' THEN a.base_quantity * a.component_quantity / (b.base_quantity * b.component_quantity) ELSE a.base_quantity * a.component_quantity END AS component_quantity,
        CASE WHEN UPPER(c.product_name) LIKE 'DUMMY%' OR UPPER(c.product_name) LIKE 'DUMY%' THEN b.base_uom ELSE a.component_uom END AS component_uom
    FROM (
        SELECT *
        FROM BOM WHERE material_no IN (SELECT material_no FROM OLALA)
    ) a
    LEFT JOIN (
        SELECT *
        FROM BOM WHERE material_no NOT IN (SELECT material_no FROM OLALA)
    ) b
        ON a.bom_component = b.bom_component
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c
        ON a.bom_component = c.product_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c1
        ON b.material_no = c1.product_id
)
SELECT 
    *,
    CASE WHEN UPPER(component_name) LIKE 'HỘP ĐỰNG%' THEN 1 ELSE 0 END AS is_box
FROM BOM_OLALA
""")